In [2]:
import os
import traceback
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np

user='hifld_summer'
pw='hifld_summer'
name='hifld_summer'
host='moria'
port=2023
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{pw}@{host}:{port}/{name}'
    )

In [3]:
sql="""
select geom, build_id, prim_occ from structures.alabama limit 500000
"""
try:
    gdf = gpd.GeoDataFrame.from_postgis(sql, con=engine,geom_col='geom') # epsg 4346

    gdf = gdf.to_crs("ESRI:102003") # USA_Contiguous_Albers_Equal_Area_Conic Projection

    gdf["new_sqmeters"] = gdf.area
    gdf["centroid"] = gdf.geometry.centroid
    buffer_distance = 250
    gdf["buffer_geom"] = gdf.centroid.buffer(buffer_distance)
    buffer_distance = str(buffer_distance)

    buffers_gdf = gdf.copy()
    centroids_gdf = gdf.copy()
    
    buffers_gdf = buffers_gdf.set_geometry('buffer_geom')
    centroids_gdf = gdf.set_geometry('centroid')

    buffers_gdf.rename(columns = {'build_id':'build_id_right'}, inplace = True)
    centroids_gdf.rename(columns = {'build_id':'build_id_left'}, inplace = True)

    
    buffers_gdf = buffers_gdf.drop(columns=['centroid', 'geom', 'new_sqmeters'])
    centroids_gdf = centroids_gdf.drop(columns=['buffer_geom', 'geom'])

    def mean_area_of_geoms_in_buffers(points, polygons, polygon_id, new_column="mean_area"):
    
        points_in_polygon = (polygons.sjoin(points, how="inner").groupby(polygon_id)["new_sqmeters"].mean().rename(new_column))
        polygons = (polygons.set_index(polygon_id).join(points_in_polygon, how="left").fillna({new_column: 0}))

        return polygons

    neighbor_gdf = mean_area_of_geoms_in_buffers(centroids_gdf, buffers_gdf, "build_id_right", new_column="mean_area_of_neighbors")

except Exception:
    print(traceback.print_exc())

count    5000.000000
mean      193.223920
std        80.926693
min        36.774637
25%       154.662151
50%       180.115342
75%       211.780538
max      2009.678497
Name: mean_area_of_neighbors, dtype: float64
